In [13]:
import time
import numpy as np
import pandas as pd

df = pd.read_csv("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=XBI&apikey=7V4FCTJVHVEY9FH5&outputsize=full&datatype=csv",
                 index_col='timestamp', usecols=['timestamp','adjusted_close'])
df = df[df['adjusted_close'] > 0]
price = df['adjusted_close'][time.strftime("%Y-%m-%d"):"2007-10-25"]

cr = (price[0]/price[-1]) - 1
pct = price.pct_change(-1)
sd = pct.std()
rfr = (1+0.03)**(1/252)-1
sr = np.sqrt(252) * ((pct.mean()-rfr)/sd)
print("IPO {} CR {} SD {} SR {}".format(df.index[-1],cr,sd,sr))

IPO 2006-02-06 CR 4.064802633316955 SD 0.01905302654120489 SR 0.54647401319181


In [14]:
tickers = ["csu.to","byd-un.to","nflx","rost","0700.hk","ccl-b.to","amzn"]
table = pd.DataFrame(index=tickers, columns=['IPO','CR','SD','SR'])

for ticker in tickers:
    df = pd.read_csv("https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol=%s&apikey=7V4FCTJVHVEY9FH5&outputsize=full&datatype=csv"%(ticker),
                     index_col='timestamp', usecols=['timestamp','adjusted_close'])
    df = df[df['adjusted_close'] > 0]
    price = df['adjusted_close'][time.strftime("%Y-%m-%d"):"2007-10-25"]
    
    cr = (price[0]/price[-1]) - 1
    pct = price.pct_change(-1)
    sd = pct.std()
    rfr = (1+0.03)**(1/252)-1
    sr = np.sqrt(252) * ((pct.mean()-rfr)/sd)
    
    table.loc[ticker] = [df.index[-1],cr,sd,sr]

table.sort_values('SR')
table

,IPO,CR,SD,SR
csu.to,2007-10-25,37.091,0.0153895,1.37647
byd-un.to,2000-01-03,67.1581,0.0214203,1.22864
nflx,2002-05-23,99.7461,0.0341701,0.99746
rost,1995-01-03,16.255,0.0178165,0.962351
0700.hk,2004-06-16,29.859,0.0241931,0.951837
ccl-b.to,2000-01-03,14.9259,0.0198439,0.872767
amzn,1997-05-15,21.3352,0.0238679,0.862987


In [ ]:
import os
df = pd.read_csv(os.path.join('data','all.csv'), index_col='date', usecols=['date','ticker','adj_close'])

In [ ]:
dfs = dict(tuple(df.groupby('ticker')))

In [ ]:
# import urllib.request

# ticker = "QQQ"
# url_string = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=%s&outputsize=full&apikey=7V4FCTJVHVEY9FH5"%(ticker)
# save = '%s.csv'%ticker  # Save data to this file

# # If you haven't already saved data, grab the data from the url
# if not os.path.exists(save):
#     with urllib.request.urlopen(url_string) as url:
#         data = json.loads(url.read().decode())
#         data = data['Time Series (Daily)']  # extract stock market data
#         # And store date, low, high, volume, close, open values to a Pandas dataframe
#         df = pd.DataFrame(columns=['Date','Low','High','Close','Open'])
#         for k,v in data.items():
#             date = dt.datetime.strptime(k, '%Y-%m-%d')
#             data_row = [date.date(),float(v['3. low']),float(v['2. high']), float(v['4. close']),float(v['1. open'])]
#             df.loc[-1,:] = data_row
#             df.index = df.index + 1
#     print('Data saved to : %s'%file_to_save)
#     df.to_csv(file_to_save)
# else:  # If data aready exists, just load from the CSV
#     print('File already exists. Loading data from CSV')
#     df = pd.read_csv(file_to_save)

In [ ]:
import matplotlib.pyplot as plt

window = 252

rolling_max = price.rolling(window, min_periods=1).max()
daily_drawdown = price/rolling_max - 1.0
max_daily_drawdown = daily_drawdown.rolling(window, min_periods=1).min()

drawdown = plt.figure(figsize=(15,8))
plt.plot(daily_drawdown)
plt.plot(max_daily_drawdown)
drawdown.suptitle('Anticipated Maximum Daily Drawdown')
plt.xlabel('Date')
plt.ylabel('Drawdown Losses in %')
plt.legend(['Actual Daily Drawdown', 'Anticipated Maximum Daily Drawdown'], loc='lower left')
plt.locator_params(nbins=10, axis='x')

cumsum = pct.cumsum()
start = np.argmax(np.maximum.accumulate(cumsum) - cumsum)
end = np.argmax(cumsum[:start])
MDP = price[end]/price[start]-1

In [ ]:
MD = plt.figure(figsize=(15,8))
plt.plot(cumsum)
plt.xlabel('Date')
plt.ylabel('Normalized Price')
plt.plot([start, end], [cumsum[start], cumsum[end]], 'o', color='Red', markersize=10)
plt.gca().invert_yaxis()
plt.locator_params(nbins=10, axis='x')

print ("Max Drawdown of {} Occured Between {} and {}".format(HDP,start,end))

In [ ]:
price = df["2018-09-04":"2007-10-25"]
pct = price.pct_change()

pd.plotting.scatter_matrix(pct, diagonal='kde', alpha=0.1,figsize=(12,12))
pct.hist(bins=50, sharex=True, figsize=(12,8))
pct.describe()

In [ ]:
from scipy import random, linalg
import seaborn as sns

matrixSize = 29
A = random.rand(matrixSize,matrixSize)
B = np.dot(A,A.transpose())


df = pd.DataFrame(B, columns=['MA','EMA','MOM','ROC','ATR','BB','PPSR',
'STOK','STO','TRIX','ADX','MACD','MassI','Vortex','KST','RSI','TSI','ACCDIST','Chaikin','MFI','OBV','FORCE','EOM','CCI',
'COPP','KELCH','ULTOSC','DONCH','STDDEV'])
corr = df.corr()

sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns)

from pandas.plotting import scatter_matrix
scatter_matrix(df,figsize=(15,9),alpha=0.3)

# # actual correlation
# corr = np.corrcoef(actual_returns, predicted_returns)[0][1]
# ax1.scatter(actual_returns, predicted_returns, color='k', marker='o', alpha=0.5, s=100)
# ax1.set_title('r = {:.2f}'.format(corr), fontsize=18)

# # shifted correlation
# shifted_actual = actual_returns[:-1]
# shifted_predicted = predicted_returns.shift(-1).dropna()
# corr = np.corrcoef(shifted_actual, shifted_predicted)[0][1]
# ax2.scatter(shifted_actual, shifted_predicted, color='k', marker='o', alpha=0.5, s=100)
# ax2.set_title('r = {:.2f}'.format(corr), fontsize=18);

# 'BBANDS','DEMA','EMA','HT_TRENDLINE','KAMA',
# 'MA','MAMA','MAVP','MIDPOINT','MIDPRICE','SAR','SAREXT','SMA','T3','TEMA','TRIMA','WMA','ADX','ADXR','APO','AROON',
# 'AROONOSC','BOP','CCI','CMO','DX','MACD','MACDEXT','MACDFIX','MFI','MINUS_DI','MINUS_DM','MOM','PLUS_DI','PLUS_DM','PPO',
# 'ROC','ROCP','ROCR','ROCR100','RSI','STOCH','STOCHF','STOCHRSI','TRIX','ULTOSC','WILLR','AD','ADOSC','OBV','HT_DCPERIOD',
# 'HT_DCPHASE','HT_PHASOR','HT_SINE','HT_TRENDMODE','AVGPRICE','MEDPRICE','TYPPRICE','WCLPRICE','ATR','NATR','TRANGE',
# '2CROWS','3BLACKCROWS','3INSIDE','3LINESTRIKE','3OUTSIDE','3STARSINSOUTH','3WHITESOLDIERS','ABANDONEDBABY','ADVANCEBLOCK',
# 'BELTHOLD','BREAKAWAY','CLOSINGMARUBOZU','CONCEALBABYSWALL','COUNTERATTACK','DARKCLOUDCOVER','DOJI','DOJISTAR',
# 'DRAGONFLYDOJI','ENGULFING','EVENINGDOJISTAR','EVENINGSTAR','GAPSIDESIDEWHITE','GRAVESTONEDOJI','HAMMER','HANGINGMAN',
# 'HARAMI','HARAMICROSS','HIGHWAVE','HIKKAKE','HIKKAKEMOD','HOMINGPIGEON','IDENTICAL3CROWS','INNECK','INVERTEDHAMMER',
# 'KICKING','KICKINGBYLENGTH','LADDERBOTTOM','LONGLEGGEDDOJI','LONGLINE','MARUBOZU','MATCHINGLOW','MATHOLD',
# 'MORNINGDOJISTAR','MORNINGSTAR','ONNECK','PIERCING','RICKSHAWMAN','RISEFALL3METHODS','SEPARATINGLINES','SHOOTINGSTAR',
# 'SHORTLINE','SPINNINGTOP','STALLEDPATTERN','STICKSANDWICH','TAKURI','TASUKIGAP','THRUSTING','TRISTAR','UNIQUE3RIVER',
# 'UPSIDEGAP2CROWS','XSIDEGAP3METHODS'
